In [72]:
#! pip install scikit-learn

In [73]:
import requests
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [74]:
# API Credentials (Replace with your credentials)
CLIENT_ID = "8d3314772d90b39ef86aeed6ff9868aa"
API_URL = "https://api.myanimelist.net/v2/anime"

In [75]:
def get_anime_data(anime_id):
    headers = {"X-MAL-Client-ID": CLIENT_ID}
    params = {"fields": "id,title,synopsis,genres,mean,rank,popularity"}
    response = requests.get(f"{API_URL}/{anime_id}", headers=headers, params=params)
    print(response)
    if response.status_code == 200:
        return response.json()
    else:
        return None

In [76]:
def fetch_anime_list(offset):
    headers = {"X-MAL-Client-ID": CLIENT_ID}
    params = {"ranking_type": "all", "limit": 100, "fields": "id,title,synopsis,genres,mean","offset":offset}
    response = requests.get(f"{API_URL}/ranking", headers=headers, params=params)
    if response.status_code == 200:
        return response.json().get("data", [])
    else:
        print(f"Error {response.status_code}: {response.text}")
        return []


In [85]:
def preprocess_data(anime_list, existing_df):
    new_data = pd.DataFrame([
        {
            "id": anime["node"]["id"], 
            "title": anime["node"]["title"], 
            "synopsis": anime["node"].get("synopsis", ""), 
            "genres": ", ".join([g["name"] for g in anime["node"].get("genres", [])]), 
            "mean": anime["node"].get("mean", 0)
        }
        for anime in anime_list
    ])
    
    if existing_df is not None:
        return pd.concat([existing_df, new_data], ignore_index=True)
    else:
        return new_data

In [78]:
def compute_similarity(df):
    vectorizer = TfidfVectorizer(stop_words="english")
    tfidf_matrix = vectorizer.fit_transform(df["synopsis"])
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return similarity_matrix

In [79]:
def recommend_anime(user_favorites, df, similarity_matrix):
    recommended = set()
    for anime_title in user_favorites:
        if anime_title in df["title"].values:
            idx = df[df["title"] == anime_title].index[0]
            similar_indices = similarity_matrix[idx].argsort()[-6:-1][::-1]
            recommended.update(df.iloc[similar_indices]["title"].values)
    return list(recommended)

In [ ]:
if __name__ == "__main__":
    df = None 
    for i in range(0, 50001, 100):
        anime_list = fetch_anime_list(i)
        df = preprocess_data(anime_list,df)
    similarity_matrix = compute_similarity(df)
    user_favorites = ["Attack on Titan", "Death Note"]
    recommendations = recommend_anime(user_favorites, df, similarity_matrix)
    print("Recommended Anime:", recommendations)

Error 504: <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<HTML><HEAD><META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=iso-8859-1">
<TITLE>ERROR: The request could not be satisfied</TITLE>
</HEAD><BODY>
<H1>504 Gateway Timeout ERROR</H1>
<H2>The request could not be satisfied.</H2>
<HR noshade size="1px">
We can't connect to the server for this app or website at this time. There might be too much traffic or a configuration error. Try again later, or contact the app or website owner.
<BR clear="all">
If you provide content to customers through CloudFront, you can find steps to troubleshoot and help prevent this error by reviewing the CloudFront documentation.
<BR clear="all">
<HR noshade size="1px">
<PRE>
Generated by cloudfront (CloudFront) HTTP3 Server
Request ID: izI5mtQ2z5M_1nn3bSE61-zNYDqz7oPPKzRPx5eAw95Q6CG_eypZ_A&#x3D;&#x3D;
</PRE>
<ADDRESS>
</ADDRESS>
</BODY></HTML>


In [88]:
print(json.dumps(anime_list, indent=2))  # Check structure


[
  {
    "node": {
      "id": 50010,
      "title": "Meitantei Conan: Hannin no Hanzawa-san",
      "main_picture": {
        "medium": "https://cdn.myanimelist.net/images/anime/1560/125527.webp",
        "large": "https://cdn.myanimelist.net/images/anime/1560/125527l.webp"
      },
      "synopsis": "Anime adaptation of the spin-off manga Meitantei Conan: Hannin no Hanzawa-san that stars the black-silhouetted \"criminal\" that appears in Detective Conan representing the mystery culprits.",
      "genres": [
        {
          "id": 4,
          "name": "Comedy"
        },
        {
          "id": 7,
          "name": "Mystery"
        },
        {
          "id": 27,
          "name": "Shounen"
        }
      ],
      "mean": 6.87
    },
    "ranking": {
      "rank": 5001
    }
  },
  {
    "node": {
      "id": 42476,
      "title": "Meitantei Conan: Shakunetsu no Ginga Tetsudou",
      "main_picture": {
        "medium": "https://cdn.myanimelist.net/images/anime/1318/108547.jp